# Lego Database

I got the idea for this analysis from [Kaggle](https://www.kaggle.com/rtatman/lego-database)

In [1]:
import sqlite3
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
conn = sqlite3.connect('lego.db')
cur = conn.cursor()

In [ ]:
# create the DataFrames with all the raw data
# sets = pd.read_csv('sets.csv')
# themes = pd.read_csv('themes.csv')

# parts = pd.read_csv('parts.csv')
# part_categories = pd.read_csv('part_categories.csv')
# colors = pd.read_csv('colors.csv')

# inventories = pd.read_csv('inventories.csv')
# inv_parts = pd.read_csv('inventory_parts.csv')
# inv_sets = pd.read_csv('inventory_sets.csv')

In [3]:
    # First I tried to add PRIMARY KEY() to all tables, but it kept raising "UNIQUE constraint failed"
    # for the different variables and tables (even though I could not find any duplicate values)
    # In this post: https://stackoverflow.com/questions/35415469/sqlite3-unique-constraint-failed-error
    # someone in the comments mentioned that PRIMARY KEY is not strictly necessary ("Usually, the primary key is generated for you by sql.")
    # so I tried removing all the PRIMARY KEY() from the tables and it worked. I don't like it, but it works.
    # READ ABOUT THIS.

### Create all tables
cur.execute("""
            CREATE TABLE IF NOT EXISTS Sets (
            Set_Num TEXT NOT NULL,
            Name TEXT,
            Year INTEGER,
            Theme_Id INTEGER,
            Num_Parts INTEGER)
            """)

cur.execute("""
            CREATE TABLE IF NOT EXISTS Themes (
            Id INTEGER NOT NULL,
            Name TEXT,
            Parent_Id INTEGER)
            """)

cur.execute("""
            CREATE TABLE IF NOT EXISTS Parts (
            Part_Num TEXT NOT NULL,
            Name TEXT,
            Part_Cat_Id INTEGER,
            Part_Material TEXT)
            """)

cur.execute("""
            CREATE TABLE IF NOT EXISTS Part_Categories (
            Id INTEGER NOT NULL,
            Name TEXT)
            """)

cur.execute("""
            CREATE TABLE IF NOT EXISTS Colors (
            Id INTEGER NOT NULL,
            Name TEXT,
            RGB TEXT,
            Is_Trans TEXT)
            """)

cur.execute("""
            CREATE TABLE IF NOT EXISTS Inventories (
            Id INTEGER NOT NULL,
            Version INTEGER,
            Set_Num TEXT)
            """)

cur.execute("""
            CREATE TABLE IF NOT EXISTS Inventory_Parts (
            Inventory_Id INTEGER NOT NULL,
            Part_Num TEXT,
            Color_Id INTEGER,
            Quantity INTEGER,
            Is_Spare TEXT)
            """)

cur.execute("""
            CREATE TABLE IF NOT EXISTS Inventory_Sets (
            Inventory_Id INTEGER NOT NULL,
            Set_Num TEXT,
            Quantity INTEGER)
            """)

conn.commit()

In [4]:
sets = pd.read_csv('sets.csv')
themes = pd.read_csv('themes.csv')
parts = pd.read_csv('parts.csv')
part_categories = pd.read_csv('part_categories.csv')
colors = pd.read_csv('colors.csv')
inventories = pd.read_csv('inventories.csv')
inv_parts = pd.read_csv('inventory_parts.csv')
inv_sets = pd.read_csv('inventory_sets.csv')

sets.to_sql('Sets', conn, if_exists='append', index=False)
themes.to_sql('Themes', conn, if_exists='append', index=False)
parts.to_sql('Parts', conn, if_exists='append', index=False)
part_categories.to_sql('Part_Categories', conn, if_exists='append', index=False)
colors.to_sql('Colors', conn, if_exists='append', index=False)
inventories.to_sql('Inventories', conn, if_exists='append', index=False)
inv_parts.to_sql('Inventory_Parts', conn, if_exists='append', index=False)
inv_sets.to_sql('Inventory_Sets', conn, if_exists='append', index=False)

conn.commit()

In [8]:
query = """
SELECT *
FROM Sets
"""

sets = pd.read_sql(query, conn)
sets

,Set_Num,Name,Year,Theme_Id,Num_Parts
0,001-1,Gears,1965,1,43
1,0011-2,Town Mini-Figures,1978,84,12
2,0011-3,Castle 2 for 1 Bonus Offer,1987,199,0
3,0012-1,Space Mini-Figures,1979,143,12
4,0013-1,Space Mini-Figures,1979,143,12
...,...,...,...,...,...
15898,XMASTREE-1,Christmas Tree,2019,410,26
15899,XWING-1,Mini X-Wing Fighter,2019,158,60
15900,XWING-2,X-Wing Trench Run,2019,158,52
15901,YODACHRON-1,Yoda Chronicles Promotional Set,2013,158,413


In [11]:
# which are the 5 sets with more parts?
query = """
SELECT Set_Num, Name, Theme_Id
    FROM Sets
    ORDER BY Num_Parts
        LIMIT 5;
"""

biggest_5_sets = pd.read_sql(query, conn)
biggest_5_sets

,Set_Num,Name,Theme_Id
0,0011-3,Castle 2 for 1 Bonus Offer,199
1,005-2,Discovery Set,366
2,00-6,Special Offer,67
3,0241401208-1,Cute Ideas,497
4,0744024471-1,100 Ways to Rebuild the World,497


In [13]:
# let's take a look at the years these sets were released
query = """
SELECT DISTINCT(Year)
FROM Sets
ORDER BY Year
"""

years = pd.read_sql(query, conn)
years

,Year
0,1949
1,1950
2,1953
3,1954
4,1955
...,...
66,2017
67,2018
68,2019
69,2020
